In [107]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt
import numpy as np
from sqlalchemy.types import String, Integer
from sqlalchemy import create_engine
from sqlalchemy import text
import psycopg2
import psycopg2.extras
import json

import geoalchemy2 
import scipy.stats as stats
import math

In [24]:
credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['database']
        try:
            db = create_engine('postgresql+psycopg2://'+db_user+':'+db_pw+'@'+host+'/'+default_db, echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(sqlcmd, args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

db, conn = pgconnect(credentials)

Connected successfully.


In [25]:
query(conn, "CREATE EXTENSION postgis;")
query(conn, "select PostGIS_Version()")


Error encountered: 
(psycopg2.errors.DuplicateObject) extension "postgis" already exists

[SQL: CREATE EXTENSION postgis;]
(Background on this error at: http://sqlalche.me/e/14/f405)


,postgis_version
0,3.3 USE_GEOS=1 USE_PROJ=1 USE_STATS=1


In [27]:
GDA2020 = gpd.read_file('SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp', crs='EPSG:4326')
GDA2020 = GDA2020[GDA2020['GCC_NAME21'] == 'Greater Sydney']
GDA2020 = GDA2020.loc[:, ['SA2_CODE21','SA2_NAME21','geometry']]

def create_wkt_element(geom, srid):
    if geom.geom_type == 'Polygon':
        geom = MultiPolygon([geom])
    return WKTElement(geom.wkt, srid)



In [28]:
GDA2020

,SA2_CODE21,SA2_NAME21,geometry
28,102011028,Avoca Beach - Copacabana,"POLYGON ((151.41373 -33.46558, 151.41362 -33.4..."
29,102011029,Box Head - MacMasters Beach,"POLYGON ((151.37484 -33.50052, 151.37507 -33.5..."
30,102011030,Calga - Kulnura,"MULTIPOLYGON (((151.20449 -33.53280, 151.20448..."
31,102011031,Erina - Green Point,"POLYGON ((151.37194 -33.43698, 151.37288 -33.4..."
32,102011032,Gosford - Springfield,"POLYGON ((151.32349 -33.42779, 151.32342 -33.4..."
...,...,...,...
637,128021537,Royal National Park,"POLYGON ((151.07363 -34.05638, 151.07360 -34.0..."
638,128021538,Sutherland - Kirrawee,"POLYGON ((151.05006 -34.02158, 151.05008 -34.0..."
639,128021607,Engadine,"POLYGON ((150.99568 -34.05361, 150.99570 -34.0..."
640,128021608,Loftus - Yarrawarrah,"POLYGON ((151.03955 -34.04175, 151.03954 -34.0..."


In [29]:
srid = 4326
GDA2020og = GDA2020.copy()  # creating a copy of the original for later
GDA2020['geom'] = GDA2020['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the function
GDA2020 = GDA2020.drop(columns="geometry")  # deleting the old copy

In [30]:
GDA2020

,SA2_CODE21,SA2_NAME21,geom
28,102011028,Avoca Beach - Copacabana,MULTIPOLYGON (((151.413733024921 -33.465580583...
29,102011029,Box Head - MacMasters Beach,MULTIPOLYGON (((151.37484081570685 -33.5005199...
30,102011030,Calga - Kulnura,MULTIPOLYGON (((151.20449037540152 -33.5328022...
31,102011031,Erina - Green Point,MULTIPOLYGON (((151.37193611462118 -33.4369790...
32,102011032,Gosford - Springfield,MULTIPOLYGON (((151.32348639265098 -33.4277852...
...,...,...,...
637,128021537,Royal National Park,MULTIPOLYGON (((151.07362997413264 -34.0563789...
638,128021538,Sutherland - Kirrawee,MULTIPOLYGON (((151.05006441218998 -34.0215774...
639,128021607,Engadine,MULTIPOLYGON (((150.99568346574816 -34.0536082...
640,128021608,Loftus - Yarrawarrah,MULTIPOLYGON (((151.03954821100714 -34.0417452...


In [31]:

statement = """
    DROP TABLE IF EXISTS GDA2020;
    CREATE TABLE GDA2020 (
        SA2_CODE21 VARCHAR(9),
        SA2_NAME21 text,
        geom GEOMETRY(MULTIPOLYGON,4326)
    );
"""


result = query(conn, statement)
print(result) 


Error encountered: 
This result object does not return rows. It has been closed automatically.
Empty DataFrame
Columns: []
Index: []


In [32]:
GDA2020.to_sql('GDA2020', conn, schema='public',method=None,if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})



373

In [33]:
#stop
scrid = 4326
stops = pd.read_csv("Stops.txt", sep=",")
stops['geom'] = gpd.points_from_xy(stops.stop_lon, stops.stop_lat)
stops['geom'] = stops['geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))

columns = ['stop_id', 'geom']
stops= stops.loc[:, columns]



print(stops)

stops.to_sql('stopschema', conn, if_exists='append',schema='public', method=None,index=False, dtype={'stop_id': String, 'geom': Geometry('POINT', srid)})


        stop_id                                        geom
0        200039   POINT (151.20666465471 -33.8822064874687)
1        200054   POINT (151.20699145565 -33.8820421431408)
2        200060  POINT (151.206292455081 -33.8840842535493)
3        201510  POINT (151.198866071817 -33.8916900512711)
4        201646  POINT (151.198881722942 -33.8933293130144)
...         ...                                         ...
114713   212753   POINT (151.07879697831 -33.8220164586429)
114714  2137185  POINT (151.116926480557 -33.8406690716775)
114715  2137186  POINT (151.116898892402 -33.8407691073139)
114716    21501  POINT (151.010576673346 -33.8139042429414)
114717  2150112  POINT (151.010481768913 -33.8139523874985)

[114718 rows x 2 columns]


718

In [34]:
conn.execute(text("""
    DROP INDEX IF EXISTS indexgda;
    CREATE INDEX indexgda ON public."GDA2020" USING GIST(geom);
"""));
conn.execute(text("""
    DROP INDEX IF EXISTS indexstops;
    CREATE INDEX indexstops ON public."GDA2020" USING GIST(geom);
"""));

In [35]:
result = query(conn, text("""
SELECT g."SA2_NAME21", COUNT(*) AS count, (COUNT(*) - avg_count) / stddev_count AS z_score
FROM public."GDA2020" g
JOIN public."stopschema" s ON ST_Contains(g.geom, s.geom)
CROSS JOIN (
SELECT AVG(count) AS avg_count,
STDDEV(count) AS stddev_count
FROM (
SELECT g."SA2_NAME21", COUNT(*) AS count
FROM public."GDA2020" g
JOIN public."stopschema" s ON ST_Contains(g.geom, s.geom)
GROUP BY g."SA2_NAME21") AS subquery) AS subquery2
GROUP BY g."SA2_NAME21", avg_count, stddev_count
ORDER BY count DESC;
"""))

stopfinal = pd.DataFrame(result, columns=["SA2_NAME21", "z_score"])
print(stopfinal)

                             SA2_NAME21   z_score
0    Dural - Kenthurst - Wisemans Ferry  6.337754
1                 Springwood - Winmalee  3.303507
2                      Katoomba - Leura  2.840263
3          Umina - Booker Bay - Patonga  2.747615
4               Campbelltown - Woodbine  2.608641
..                                  ...       ...
367                         Chippendale -1.595296
368                         Wolli Creek -1.595296
369                      Badgerys Creek -1.618458
370              Blue Mountains - North -1.687944
371              Blue Mountains - South -1.711107

[372 rows x 2 columns]


In [36]:
#polls
polls = pd.read_csv('PollingPlaces2019.csv', usecols = ['polling_place_id','the_geom','longitude','latitude'])
polls.dropna(subset=['the_geom'], inplace=True)
polls['geom'] = gpd.points_from_xy(polls.longitude, polls.latitude)
polls['geom'] = polls['geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))
polls = polls.loc[:, ['polling_place_id','geom']]


print(polls)



      polling_place_id                             geom
13                  58         POINT (151.081 -33.9847)
15                 392         POINT (150.817 -33.7475)
16                  31  POINT (151.1148974 -33.9767897)
17                  67         POINT (151.111 -33.9756)
18               56500         POINT (151.075 -33.9413)
...                ...                              ...
2924              2810      POINT (150.85177 -34.54724)
2925              2809         POINT (150.858 -34.5642)
2926             58798  POINT (150.8597546 -34.5508228)
2927             31242         POINT (150.424 -34.4409)
2928               564         POINT (150.866 -34.5316)

[2790 rows x 2 columns]


In [37]:


statement = """
DROP TABLE IF EXISTS polls;
CREATE TABLE polls (
    polling_place_id integer,
    geom GEOMETRY(POINT,4326));
"""


result = query(conn, statement)
polls.to_sql('polls', conn, if_exists='append', index=False, dtype={'polling_place_id': Integer, 'geom': Geometry('POINT', srid)})


Error encountered: 
This result object does not return rows. It has been closed automatically.


790

In [38]:
result = query(conn, text("""
WITH subquery AS (
    SELECT g."SA2_NAME21", COUNT(*) AS count
    FROM public."GDA2020" G
    JOIN public."polls" p ON ST_Contains(G.geom, p.geom)
    GROUP BY g."SA2_NAME21"
)
SELECT subquery."SA2_NAME21", subquery.count, (subquery.count - avg_value) / std_deviation AS z_score
FROM subquery
CROSS JOIN (
    SELECT AVG(subquery.count) AS avg_value, STDDEV(subquery.count) AS std_deviation
    FROM subquery
) AS subquery2
ORDER BY subquery.count DESC;

"""))

pollfinal = pd.DataFrame(result, columns=["SA2_NAME21", "z_score"])

print(pollfinal)

                         SA2_NAME21    z_score
0    Sydney (North) - Millers Point  14.476684
1        Sydney (South) - Haymarket   5.504492
2                Parramatta - North   3.322067
3                  Chatswood - East   2.837084
4                           Penrith   2.109609
..                              ...        ...
348                         Pyrmont  -0.800291
349              Canterbury - South  -0.800291
350                          Putney  -0.800291
351                  Bexley - North  -0.800291
352                          Berala  -0.800291

[353 rows x 2 columns]


In [39]:
school_future = gpd.read_file('catchments/catchments_future.shp')
school_primary = gpd.read_file('catchments/catchments_primary.shp')
school_secondary = gpd.read_file('catchments/catchments_secondary.shp')

datasets = [school_future,school_primary,school_secondary]

scrid = 4326
for i in range(len(datasets)):
    datasets[i].columns = datasets[i].columns.str.strip()
    datasets[i].dropna(inplace=True)
    datasets[i]['geom'] = datasets[i]['geometry'].apply(lambda x: create_wkt_element(geom=x, srid=srid))


In [40]:
school_future = school_future[['geom']]
school_primary = school_primary[['geom']]
school_secondary = school_secondary[['geom']]
print(school_secondary)

                                                  geom
33   MULTIPOLYGON (((151.2098245099502 -33.85422949...
131  MULTIPOLYGON (((150.75452293778994 -33.9547194...
170  MULTIPOLYGON (((150.87407875014713 -33.6566449...
246  MULTIPOLYGON (((150.86956491473543 -33.7101137...
380  MULTIPOLYGON (((149.2408471104619 -34.91439387...
381  MULTIPOLYGON (((149.20599879911867 -35.3710405...


In [94]:
ypopulation = pd.read_csv('Population.csv')
ypopulation['young_people'] = ypopulation['0-4_people'] + ypopulation['5-9_people'] + ypopulation['10-14_people'] + ypopulation['15-19_people']
ypopulation = ypopulation.loc[:, ['sa2_name','young_people']]

print(ypopulation)

                        sa2_name  young_people
0       Avoca Beach - Copacabana          2121
1    Box Head - MacMasters Beach          2471
2                Calga - Kulnura           961
3            Erina - Green Point          3205
4          Gosford - Springfield          4364
..                           ...           ...
368          Royal National Park            20
369        Sutherland - Kirrawee          5078
370                     Engadine          5118
371         Loftus - Yarrawarrah          2073
372             Woronora Heights           965

[373 rows x 2 columns]


In [42]:
statement = """
DROP TABLE IF EXISTS primarys;
CREATE TABLE primarys (
    geom GEOMETRY(MULTIPOLYGON,4326));
"""
result = query(conn, statement)
school_primary.to_sql('primarys', conn, schema='public',method=None,if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})


Error encountered: 
This result object does not return rows. It has been closed automatically.


4

In [43]:
statement = """
DROP TABLE IF EXISTS secondarys;
CREATE TABLE secondarys (
    geom GEOMETRY(MULTIPOLYGON,4326));
"""
result = query(conn, statement)

school_secondary.to_sql('secondarys', conn, schema='public',method=None,if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})


Error encountered: 
This result object does not return rows. It has been closed automatically.


6

In [61]:

statement = """
DROP TABLE IF EXISTS future;
CREATE TABLE future (
    geom GEOMETRY(MULTIPOLYGON,4326));
"""
result = query(conn, statement)
school_future.to_sql('future', conn, schema='public',method=None,if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})


Error encountered: 
This result object does not return rows. It has been closed automatically.


30

In [86]:
result = query(conn, text("""
    SELECT g."SA2_NAME21", COUNT(*)
    FROM public."GDA2020" g
        JOIN public."primarys" p ON ST_Intersects(g.geom, p.geom)
    GROUP BY g."SA2_NAME21"
    ORDER BY count DESC
"""))
primdata = pd.DataFrame(result, columns=["SA2_NAME21", "count"])



In [87]:
result = query(conn, text("""
    SELECT g."SA2_NAME21", COUNT(*)
    FROM public."GDA2020" g
        JOIN public."secondarys" p ON ST_Intersects(g.geom, p.geom)
    GROUP BY g."SA2_NAME21"
    ORDER BY count DESC
"""))

print(result)

seconddata = pd.DataFrame(result, columns=["SA2_NAME21", "count"])


                         SA2_NAME21  count
0       Schofields (West) - Colebee      4
1                      Quakers Hill      4
2      Kellyville Ridge - The Ponds      4
3      Blacktown (North) - Marayong      4
4                 Schofields - East      4
5                 Box Hill - Nelson      2
6           Camperdown - Darlington      2
7                   Centennial Park      2
8                       Chippendale      2
9      Claymore - Eagle Vale - Raby      2
10             Cobbitty - Bringelly      2
11                     Darlinghurst      2
12       Double Bay - Darling Point      2
13  Gledswood Hills - Gregory Hills      2
14          Glendenning - Dean Park      2
15                         Glenwood      2
16                  Harrington Park      2
17         Hassall Grove - Plumpton      2
18                 Kensington (NSW)      2
19       Lalor Park - Kings Langley      2
20     Leppington - Catherine Field      2
21        Lethbridge Park - Tregear      2
22       Ma

In [88]:
result = query(conn, text("""
    SELECT g."SA2_NAME21", COUNT(*)
    FROM public."GDA2020" g
        JOIN public."future" p ON ST_Intersects(g.geom, p.geom)
    GROUP BY g."SA2_NAME21"
    ORDER BY count DESC
"""))


futuredata = pd.DataFrame(result, columns=["SA2_NAME21", "count"])

print(result)

                          SA2_NAME21  count
0    Gledswood Hills - Gregory Hills     12
1                            St Ives      8
2                   Kensington (NSW)      8
3                   Randwick - North      8
4                           Waterloo      8
..                               ...    ...
98                       Surry Hills      2
99      Sydenham - Tempe - St Peters      2
100   Toongabbie - Constitution Hill      2
101                Toongabbie - West      2
102   West Hoxton - Middleton Grange      2

[103 rows x 2 columns]


In [133]:
ypopulation = ypopulation.rename(columns={'sa2_name': 'SA2_NAME21'})

concatdf = pd.concat([futuredata, seconddata, primdata], ignore_index=True)

concatdf = concatdf.groupby('SA2_NAME21')['count'].sum().reset_index()

concatdf = pd.concat([ypopulation, concatdf], ignore_index=True)

concatdf = concatdf.groupby('SA2_NAME21').sum().reset_index()

concatdf['rate'] = (concatdf['count'] / concatdf['young_people']) * 1000
concatdf = concatdf.drop(concatdf[concatdf['rate'] == 0].index)
concatdf = concatdf.drop(concatdf[concatdf['young_people'] == 0].index)

concatdf = concatdf.reset_index(drop=True)
pd.set_option("display.max_rows", None)  # To display all rows
pd.set_option("display.max_columns", None)  # To display all columns
concatdf['z scores'] = stats.zscore(concatdf['rate'])
print(concatdf)




                                      SA2_NAME21  young_people  count  \
0                                 Acacia Gardens        1062.0    2.0   
1                                Annandale (NSW)        1947.0    2.0   
2                    Arncliffe - Bardwell Valley        3501.0    2.0   
3                          Asquith - Mount Colah        6041.0    2.0   
4                            Austral - Greendale        3527.0    2.0   
5                                 Badgerys Creek           1.0    2.0   
6                                        Balmain        3365.0    2.0   
7                                  Bellevue Hill        3188.0    6.0   
8                                        Belrose        2254.0    4.0   
9                     Berowra - Brooklyn - Cowan        3318.0    2.0   
10                 Bidwill - Hebersham - Emerton        6153.0    2.0   
11                 Blacktown (East) - Kings Park        3908.0    2.0   
12                  Blacktown (North) - Marayong   

In [162]:
business = pd.read_csv('Businesses.csv')
retail = business.loc[business['industry_name'] == 'Retail Trade', ['industry_name', 'sa2_code', 'total_businesses']]
health = business.loc[business['industry_name'] == 'Health Care and Social Assistance', ['industry_name', 'sa2_code','total_businesses']]
population = pd.read_csv('Population.csv', usecols = ['sa2_code', 'sa2_name','total_people'])

print(population)




      sa2_code                                     sa2_name  total_people
0    102011028                     Avoca Beach - Copacabana          7530
1    102011029                  Box Head - MacMasters Beach         11052
2    102011030                              Calga - Kulnura          4748
3    102011031                          Erina - Green Point         14803
4    102011032                        Gosford - Springfield         21346
5    102011033                                      Kariong          6518
6    102011034                  Kincumber - Picketts Valley          7628
7    102011035                                       Narara          7191
8    102011036                       Niagara Park - Lisarow          8237
9    102011037                      Point Clare - Koolewong          6575
10   102011038                         Saratoga - Davistown          7179
11   102011039                       Terrigal - North Avoca         14890
12   102011040                 Umina -

In [172]:
#population

statement = """
DROP TABLE IF EXISTS population;
CREATE TABLE population (
    sa2_code integer,
    sa2_name text,
    total_people integer   
    );
"""

result = query(conn, statement)
population.to_sql('population', conn, if_exists='append',schema='public', method=None,index=False)


Error encountered: 
This result object does not return rows. It has been closed automatically.


373

In [178]:
#health

statement = """
DROP TABLE IF EXISTS health;
CREATE TABLE health (
    industry_name text,
    sa2_code integer,
    total_businesses integer   
    );
"""
result = query(conn, statement)

health.to_sql('health', conn, if_exists='append',schema='public', method=None,index=False)


Error encountered: 
This result object does not return rows. It has been closed automatically.


643

In [191]:
#retail

statement = """
DROP TABLE IF EXISTS retail;
CREATE TABLE retail (
    industry_name text,
    sa2_code integer,
    total_businesses integer   
    );
"""
result = query(conn, statement)

retail.to_sql('retail', conn, if_exists='append',schema='public', method=None,index=False)




Error encountered: 
This result object does not return rows. It has been closed automatically.


643

In [203]:
#retail z score
result = query(conn, text("""
SELECT sa2_name, ROUND(CAST(r.total_businesses AS DECIMAL(10,2))/CAST(total_people AS DECIMAL(10,2)) * 1000, 5) as rper1000
FROM public."retail" r
INNER JOIN public."population" p ON p."sa2_code" = r."sa2_code"
WHERE total_people != 0;
"""))

retailz = pd.DataFrame(result, columns=["sa2_name", "rper1000"])
retailz['z scores health'] = stats.zscore(retailz["rper1000"])
print(retailz)


                                        sa2_name    rper1000  z scores health
0                       Avoca Beach - Copacabana     5.97610        -0.109739
1                    Box Head - MacMasters Beach     4.52407        -0.115541
2                                Calga - Kulnura    12.00505        -0.085650
3                            Erina - Green Point    10.26819        -0.092589
4                          Gosford - Springfield     8.43249        -0.099924
5                                        Kariong     3.06843        -0.121357
6                    Kincumber - Picketts Valley     5.50603        -0.111617
7                                         Narara     1.80782        -0.126394
8                         Niagara Park - Lisarow     3.52070        -0.119550
9                        Point Clare - Koolewong     3.04183        -0.121463
10                          Saratoga - Davistown     4.31815        -0.116364
11                        Terrigal - North Avoca     4.96978    

In [202]:
#health z score
result = query(conn, text("""
SELECT sa2_name, ROUND(CAST(h.total_businesses AS DECIMAL(10,2))/CAST(total_people AS DECIMAL(10,2)) * 1000, 5) as hper1000
FROM public.health h
INNER JOIN public.population p ON p.sa2_code = h.sa2_code
WHERE total_people != 0;
"""))

healthz = pd.DataFrame(result, columns=["sa2_name", "hper1000"])
healthz['z scores health'] = stats.zscore(healthz["hper1000"])
print(healthz)

                                        sa2_name    hper1000  z scores health
0                       Avoca Beach - Copacabana     9.82736        -0.047607
1                    Box Head - MacMasters Beach     4.97647        -0.079456
2                                Calga - Kulnura     9.05644        -0.052668
3                            Erina - Green Point    13.71344        -0.022092
4                          Gosford - Springfield    14.19470        -0.018932
5                                        Kariong     3.37527        -0.089969
6                    Kincumber - Picketts Valley     7.73466        -0.061347
7                                         Narara     3.33751        -0.090217
8                         Niagara Park - Lisarow     2.67088        -0.094594
9                        Point Clare - Koolewong     5.32319        -0.077180
10                          Saratoga - Davistown     3.62167        -0.088351
11                        Terrigal - North Avoca    11.61854    